# Microlensing Analysis Tutorial

This tutorial demonstrates how to analyze microlensing events using popular fitting tools and integrate your results with `microlens-submit` for submission management.

## What You'll Learn

- Setting up your analysis environment
- Loading and preparing microlensing data
- Fitting events with MulensModel and pyLIMA
- Integrating your results with microlens-submit
- Best practices for reproducible analysis

## Prerequisites

- Python 3.8 or higher
- Basic familiarity with Python and Jupyter notebooks
- Understanding of microlensing concepts

---

## 1. Environment Setup

First, let's set up our analysis environment with the required packages:

In [ ]:
# Install required packages
!pip install microlens-submit MulensModel pyLIMA numpy matplotlib pandas scipy astropy

In [ ]:
# Import standard libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

# Import microlensing tools
import MulensModel as mm
import microlens_submit

# Import astropy for coordinate handling
import astropy.units as u
from astropy.coordinates import SkyCoord

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("✅ Environment setup complete!")

## 2. Data Preparation

Let's create some example microlensing data for demonstration purposes. In practice, you would load your actual event data:

In [ ]:
# Create example light curve data
def generate_example_lightcurve(t0=2459123.5, u0=0.1, tE=20.0, baseline_mag=18.0, noise=0.02):
    """Generate a simple single-lens microlensing light curve."""
    # Time range around the event
    t = np.linspace(t0 - 2*tE, t0 + 2*tE, 100)
    
    # Calculate magnification
    u = np.sqrt(u0**2 + ((t - t0)/tE)**2)
    A = (u**2 + 2) / (u * np.sqrt(u**2 + 4))
    
    # Calculate magnitude
    mag = baseline_mag - 2.5 * np.log10(A)
    
    # Add noise
    mag += np.random.normal(0, noise, len(mag))
    mag_err = np.full_like(mag, noise)
    
    return pd.DataFrame({
        'time': t,
        'magnitude': mag,
        'magnitude_error': mag_err
    })

# Generate example data
example_data = generate_example_lightcurve()

# Save to file for demonstration
example_data.to_csv('example_event_W149.txt', sep=' ', index=False, header=False)

print("✅ Example light curve data generated")
print(f"Data shape: {example_data.shape}")
example_data.head()

In [ ]:
# Plot the example light curve
plt.figure(figsize=(12, 6))
plt.errorbar(example_data['time'], example_data['magnitude'], 
            yerr=example_data['magnitude_error'], fmt='o', alpha=0.7, label='Data')
plt.xlabel('Time (JD)')
plt.ylabel('Magnitude')
plt.title('Example Microlensing Light Curve')
plt.gca().invert_yaxis()
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3. Fitting with MulensModel

Now let's fit our example data using MulensModel:

In [ ]:
# Load the data with MulensModel
data = mm.MulensData(
    file_name='example_event_W149.txt',
    phot_fmt='mag',
    plot_properties={'color': 'blue', 'label': 'W149'}
)

print("✅ Data loaded with MulensModel")
print(f"Number of data points: {len(data.time)}")
print(f"Time range: {data.time.min():.1f} to {data.time.max():.1f} JD")

In [ ]:
# Create initial model parameters
initial_params = {
    't_0': 2459123.5,  # Time of closest approach
    'u_0': 0.1,       # Impact parameter
    't_E': 20.0,      # Einstein crossing time
}

# Create the model
model = mm.Model(initial_params)

# Create the event
event = mm.Event(datasets=data, model=model)

print("✅ Model and event created")
print(f"Initial parameters: {initial_params}")

In [ ]:
# Plot the initial fit
plt.figure(figsize=(12, 6))
event.plot_data()
event.plot_model()
plt.xlabel('Time (JD)')
plt.ylabel('Magnitude')
plt.title('Initial Model Fit')
plt.gca().invert_yaxis()
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Perform the fit
print("Fitting model...")
start_time = time.time()

# Fit the model
event.fit()

fit_time = time.time() - start_time
print(f"✅ Fit completed in {fit_time:.2f} seconds")

# Get best-fit parameters
best_params = event.model.parameters.parameters
print(f"\nBest-fit parameters:")
for param, value in best_params.items():
    print(f"  {param}: {value:.6f}")

# Calculate chi-squared
chi2 = event.get_chi2()
print(f"\nChi-squared: {chi2:.2f}")
print(f"Degrees of freedom: {len(data.time) - len(best_params)}")
print(f"Reduced chi-squared: {chi2 / (len(data.time) - len(best_params)):.2f}")

In [ ]:
# Plot the final fit
plt.figure(figsize=(12, 6))
event.plot_data()
event.plot_model()
plt.xlabel('Time (JD)')
plt.ylabel('Magnitude')
plt.title('Best-Fit Model')
plt.gca().invert_yaxis()
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 4. Integration with microlens-submit

Now let's integrate our fitting results with `microlens-submit` to manage our submission:

In [ ]:
# Initialize or load a submission project
project_path = Path("./my_analysis_submission")
submission = microlens_submit.load(project_path)

# Set team information
submission.team_name = "Tutorial Team"
submission.tier = "intermediate"

# Add hardware information for reproducibility
submission.hardware_info = {
    "platform": "Local Analysis",
    "python_version": "3.10.0",
    "packages": {
        "MulensModel": "3.2.0",
        "numpy": "1.24.0",
        "scipy": "1.10.0"
    }
}

print(f"✅ Submission project initialized at: {project_path}")

In [ ]:
# Get or create an event
event_id = "example-event-001"
event_obj = submission.get_event(event_id)

# Add our fitted solution
solution = event_obj.add_solution(
    model_type="single_lens",
    parameters=best_params
)

# Add metadata about the solution
solution.notes = "Single-lens fit using MulensModel with chi-squared minimization"
solution.log_likelihood = -chi2 / 2  # Approximate log-likelihood
solution.n_data_points = len(data.time)
solution.used_astrometry = False
solution.used_postage_stamps = False

# Record computational information
solution.set_compute_info(
    cpu_hours=fit_time / 3600,  # Convert seconds to hours
    wall_time_hours=fit_time / 3600
)

print(f"✅ Solution added: {solution.solution_id}")
print(f"Model type: {solution.model_type}")
print(f"Parameters: {solution.parameters}")

In [ ]:
# Save the submission
submission.save()
print("✅ Submission saved")

# List all solutions for this event
print(f"\nSolutions for {event_id}:")
for sol in event_obj.get_active_solutions():
    print(f"  ID: {sol.solution_id[:8]}...")
    print(f"    Model: {sol.model_type}")
    print(f"    Log-likelihood: {sol.log_likelihood:.2f}")
    print(f"    Active: {sol.is_active}")
    print()

## 5. Advanced Example: Binary Lens Fitting

Let's demonstrate a more complex binary lens fit:

In [ ]:
# Generate example binary lens data
def generate_binary_lightcurve(t0=2459123.5, u0=0.1, tE=20.0, q=0.001, s=1.2, alpha=45.0):
    """Generate a binary lens microlensing light curve."""
    # This is a simplified example - real binary lens calculations are more complex
    t = np.linspace(t0 - 2*tE, t0 + 2*tE, 150)
    
    # Simplified binary lens magnification (this is not accurate but for demonstration)
    u = np.sqrt(u0**2 + ((t - t0)/tE)**2)
    
    # Add some binary lens features (caustic crossing effects)
    binary_factor = 1.0 + 0.5 * np.exp(-((t - t0)/tE)**2) * np.sin(alpha * np.pi/180)
    A = (u**2 + 2) / (u * np.sqrt(u**2 + 4)) * binary_factor
    
    # Calculate magnitude
    baseline_mag = 18.0
    mag = baseline_mag - 2.5 * np.log10(A)
    
    # Add noise
    noise = 0.02
    mag += np.random.normal(0, noise, len(mag))
    mag_err = np.full_like(mag, noise)
    
    return pd.DataFrame({
        'time': t,
        'magnitude': mag,
        'magnitude_error': mag_err
    })

# Generate binary lens data
binary_data = generate_binary_lightcurve()
binary_data.to_csv('example_binary_event_W149.txt', sep=' ', index=False, header=False)

print("✅ Binary lens example data generated")

In [ ]:
# Load binary lens data
binary_mm_data = mm.MulensData(
    file_name='example_binary_event_W149.txt',
    phot_fmt='mag',
    plot_properties={'color': 'red', 'label': 'Binary W149'}
)

# Create binary lens model
binary_params = {
    't_0': 2459123.5,
    'u_0': 0.1,
    't_E': 20.0,
    'q': 0.001,      # Mass ratio
    's': 1.2,       # Separation
    'alpha': 45.0   # Source trajectory angle
}

binary_model = mm.Model(binary_params)
binary_event = mm.Event(datasets=binary_mm_data, model=binary_model)

print("✅ Binary lens model created")

In [ ]:
# Fit binary lens model
print("Fitting binary lens model...")
start_time = time.time()

binary_event.fit()

binary_fit_time = time.time() - start_time
print(f"✅ Binary lens fit completed in {binary_fit_time:.2f} seconds")

# Get best-fit parameters
binary_best_params = binary_event.model.parameters.parameters
binary_chi2 = binary_event.get_chi2()

print(f"\nBinary lens best-fit parameters:")
for param, value in binary_best_params.items():
    print(f"  {param}: {value:.6f}")
print(f"Chi-squared: {binary_chi2:.2f}")

In [ ]:
# Add binary lens solution to submission
binary_event_id = "example-binary-event-001"
binary_event_obj = submission.get_event(binary_event_id)

binary_solution = binary_event_obj.add_solution(
    model_type="binary_lens",
    parameters=binary_best_params
)

# Add physical parameters (example values)
binary_solution.physical_parameters = {
    "M_L": 0.5,      # Lens mass (M☉)
    "D_L": 6.0,      # Lens distance (kpc)
    "M_planet": 1.7, # Planet mass (M⊕)
    "a": 2.5         # Semi-major axis (AU)
}

binary_solution.notes = "Binary lens fit with planetary companion using MulensModel"
binary_solution.log_likelihood = -binary_chi2 / 2
binary_solution.n_data_points = len(binary_mm_data.time)

binary_solution.set_compute_info(
    cpu_hours=binary_fit_time / 3600,
    wall_time_hours=binary_fit_time / 3600
)

print(f"✅ Binary lens solution added: {binary_solution.solution_id}")

## 6. Model Comparison and Selection

Let's compare our different models and select the best one:

In [ ]:
# Compare models using likelihood
print("Model Comparison:")
print("=" * 50)

models = [
    ("Single Lens", solution.log_likelihood, len(best_params)),
    ("Binary Lens", binary_solution.log_likelihood, len(binary_best_params))
]

for name, log_likelihood, n_params in models:
    # Calculate BIC (Bayesian Information Criterion)
    n_data = len(data.time)  # Assuming same number of data points
    bic = -2 * log_likelihood + n_params * np.log(n_data)
    
    print(f"{name}:")
    print(f"  Log-likelihood: {log_likelihood:.2f}")
    print(f"  Parameters: {n_params}")
    print(f"  BIC: {bic:.2f}")
    print()

# Select the model with lower BIC (better fit)
best_model = min(models, key=lambda x: -2 * x[1] + x[2] * np.log(len(data.time)))
print(f"Best model by BIC: {best_model[0]}")

In [ ]:
# Deactivate the worse model (optional)
if best_model[0] == "Binary Lens":
    solution.deactivate()
    print("Deactivated single lens solution")
else:
    binary_solution.deactivate()
    print("Deactivated binary lens solution")

# Save updated submission
submission.save()
print("✅ Updated submission saved")

## 7. Final Export

When you're ready to submit your results:

In [ ]:
# Export the final submission
submission.export(filename="tutorial_final_submission.zip")
print("✅ Final submission exported!")

# Check what's included
import zipfile
with zipfile.ZipFile("tutorial_final_submission.zip", 'r') as zip_ref:
    print("\nContents of submission package:")
    for file in zip_ref.namelist():
        print(f"  {file}")

## 8. Best Practices and Tips

### Reproducibility
- Always record your computational environment
- Use version control for your analysis scripts
- Document your parameter choices and assumptions

### Model Selection
- Compare models using statistical criteria (BIC, AIC)
- Consider physical plausibility, not just statistical fit
- Use multiple fitting codes when possible

### Data Quality
- Check for systematic errors in your data
- Validate your model assumptions
- Consider the impact of blending and finite source effects

### Workflow Management
- Save intermediate results frequently
- Use descriptive names for your solutions
- Keep a log of your analysis decisions

---

## Next Steps

1. **Explore more advanced models** (parallax, lens orbital motion, finite source effects)
2. **Try other fitting codes** (pyLIMA, VBMicrolensing, BAGEL)
3. **Analyze real data** from the microlensing data challenge
4. **Join the community** for questions and collaboration

Happy analyzing! 🪐